In [1]:
import pandas as pd
import nltk
import ast
import numpy as np
import os
import ast
from urllib.request import urlopen
from bs4 import BeautifulSoup
import os.path
nltk.download('stopwords')
nltk.download('words')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/domantas/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /home/domantas/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to /home/domantas/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:

from sklearn.externals import joblib
filename = "Models/LR_model.joblib"
lr = joblib.load(filename) 
lr_max = joblib.load("Models/LR_maxtrain.joblib") 

import pickle
pickle_in = open("Models/word_frequency.picle","rb")
words_frequency = pickle.load(pickle_in)



In [ ]:
urls = 'https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.coo_matrix.html'
top = 2500

In [ ]:
for url in urls:
    html = urlopen(url, timeout=15).read()
    soup = BeautifulSoup(html, "html.parser")
    [tag.decompose() for tag in soup("script")]
    [tag.decompose() for tag in soup("style")]
    text = soup.get_text()
    lines = (line.strip() for line in text.splitlines())
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    text = '\n'.join(chunk.lower() for chunk in chunks if chunk)
    tokens = nltk.word_tokenize(text)
    
    from collections import Counter
    counter = 0
    features_pred = np.zeros(top * len(words_frequency)).reshape(len(words_frequency), top)
    c = [word for word, word_count in Counter(tokens).most_common(top)]
    
    for category in words_frequency.keys():
        for word in c:
            if word in words_frequency[category]:
                features_pred[counter][words_frequency[category].index(word)] = 1
        counter+=1
        
    category_weight = []
    for i in features_pred:
        weight_cof = np.where(i == 1)[0]
        weight_sum = 0
        for cof in weight_cof:
            weight_sum += top - cof
        category_weight.append(weight_sum)
    cat_index = category_weight.index(max(category_weight))
    category = list(words_frequency.keys())[cat_index]
    feature = features_pred[cat_index].reshape(-1, top)
    print(category)
    
    prediction = lr.predict(feature)
    print("Predicted category : ", list(words_frequency.keys())[int(prediction[0])])
    